# Tarea 3 MongoDB, arXiv
##### Integrantes:
Bruno Morici, ROL: 202373555-8,
Martin Aranda, ROL: 202373021-1

## Primera Etapa
Instalación de dependencias

In [6]:
# Instalamos dependencias
!pip install pymongo
!pip install tqdm


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Segunda Etapa
Conexión a la BD "arxiv_db" y creación de colección "articles"

In [8]:
from pymongo import MongoClient

# Conexión a Replica Set con preferencia al primario
client = MongoClient("mongodb://mongo1:27017,mongo2:27017,mongo3:27017/?replicaSet=rs0")

db = client["arxiv_db"]
collection = db["articles"]

try:
    server_info = client.server_info()
    print("✅ Conexión exitosa")
    print(f"Primario: {client.primary}") # El cliente retorna un primario mal detectado, es normal, pero las escrituras se realizan sobre mongo1
except Exception as e:
    print(f"❌ Error de conexión: {e}")

✅ Conexión exitosa
Primario: ('mongo3', 27017)


## Tercera Etapa
Procesamiento del dataset hacia la BD (ejecutar una sola vez, luego los datos quedan en la BD)

Observación: El dataset tiene un JSON en cada línea, no es una lista de JSON's. Recorremos cada JSON, lo parseamos y guardamos en una lista de data (demora unos 6 minutos)

In [ ]:
import json
from tqdm import tqdm # Para mostrar una barra de progreso

# Cargar y procesar el archivo JSON
print("Iniciando carga de datos a MongoDB...")

# Carga por lotes de 1000 en 1000
def load_data_batch(batch_size=1000):
    batch = []
    total_docs = 0
    
    # Primero contamos las lineas para la barra de progreso
    with open('arxiv-metadata-oai-snapshot.json', 'r', encoding='utf-8') as f:
        total_lines = sum(1 for _ in f)
    
    # Insertamos los datos en el batch
    with open('arxiv-metadata-oai-snapshot.json', 'r', encoding='utf-8') as f:
        for line in tqdm(f, total=total_lines, desc="Cargando datos"):
            batch.append(json.loads(line)) # Parseamos la linea a un JSON
            
            # Si el batch excede el maximo, lo vaciamos e insertamos los datos en la BD
            if len(batch) >= batch_size:
                collection.insert_many(batch)
                total_docs += len(batch)
                batch = []
        
        # Insertar el ultimo lote, en caso de que el ultimo lote sean menos de 1000
        if batch:
            collection.insert_many(batch)
            total_docs += len(batch)
    
    print(f"\nTotal de documentos insertados: {total_docs}")

load_data_batch() # Llamamos la funcion creada anteriormente

# 3. Verificacion final
print("\nResumen de la base de datos:")
print(f"- Colección: {collection.name}")    
print(f"- Ejemplo de documento: {collection.find_one()}")


Iniciando carga de datos a MongoDB...


Cargando datos: 100%|██████████| 2744489/2744489 [28:58<00:00, 1578.86it/s] 



Total de documentos insertados: 2744489

Resumen de la base de datos:
- Colección: articles


_OperationCancelled: operation cancelled

In [12]:
ports = [27017, 27018, 27019]
for i, port in enumerate(ports, start=1):
    client = MongoClient(f"mongodb://localhost:{port}")
    count = client["arxiv_db"]["articles"].estimated_document_count()
    print(f"📦 Documentos estimados en mongo{i}: {count}")

📦 Documentos estimados en mongo1: 2744489
📦 Documentos estimados en mongo2: 2744489
📦 Documentos estimados en mongo3: 2744489


## Cuarta etapa: Consultas

Consulta test

In [4]:
def buscar_software(port, nombre):
    try:
        client = MongoClient(f"mongodb://localhost:{port}", serverSelectionTimeoutMS=5000)
        db = client["arxiv_db"]
        collection = db["articles"]

        resultados = collection.find(
            {"title": {"$regex": "Software", "$options": "i"}},
            {"_id": 0, "title": 1}
        ).limit(3)  # solo para mostrar pocos

        print(f"🟢 {nombre} ({port}) - Artículos con 'Software':")
        encontrados = False
        for doc in resultados:
            print(f"   🔹 {doc['title']}")
            encontrados = True
        if not encontrados:
            print("   ⚠️ No se encontraron artículos.")
        print("-" * 50)
    except Exception as e:
        print(f"🔴 Error en {nombre} ({port}): {e}")

# Ejecutar búsqueda en los tres nodos
buscar_software(27017, "mongo1")
buscar_software(27018, "mongo2")
buscar_software(27019, "mongo3")


🟢 mongo1 (27017) - Artículos con 'Software':
   🔹 Solar Magnetic Tracking. I. Software Comparison and Recommended
  Practices
   🔹 On How Developers Test Open Source Software Systems
   🔹 Experiences of Engineering Grid-Based Medical Software
--------------------------------------------------
🟢 mongo2 (27018) - Artículos con 'Software':
   🔹 Solar Magnetic Tracking. I. Software Comparison and Recommended
  Practices
   🔹 On How Developers Test Open Source Software Systems
   🔹 Experiences of Engineering Grid-Based Medical Software
--------------------------------------------------
🟢 mongo3 (27019) - Artículos con 'Software':
   🔹 Solar Magnetic Tracking. I. Software Comparison and Recommended
  Practices
   🔹 On How Developers Test Open Source Software Systems
   🔹 Experiences of Engineering Grid-Based Medical Software
--------------------------------------------------


Consulta 1

In [ ]:
from datetime import datetime

def leer_articulos_2025(collection, client):
    try:
        primary_info = client.primary
        print(f"📦 Leyendo datos desde: {primary_info[0]}:{primary_info[1]} (PRIMARY)")

        articulos_2025 = []

        cursor = collection.find(
            {"versions.created": {"$exists": True}},
            {"_id": 0, "title": 1, "versions": 1}
        )

        for doc in cursor:
            versions = doc.get("versions", [])
            fecha = None

            for v in versions:
                fecha_str = v.get("created")
                if fecha_str:
                    try:
                        # Intenta parsear fecha (ajusta el formato según tus datos)
                        fecha = datetime.strptime(fecha_str, "%a, %d %b %Y %H:%M:%S %Z")
                    except Exception as e:
                        # Si no pudo parsear, omite esa versión
                        continue
                    if fecha.year == 2025:
                        articulos_2025.append({
                            "titulo": doc.get("title", "Sin título"),
                            "fecha_creacion": fecha
                        })
                        break  # Sólo la primera versión con fecha 2025 importa

            if len(articulos_2025) >= 20:
                break

        print(f"🔎 Se encontraron {len(articulos_2025)} artículos con versión creada en 2025:")
        for art in articulos_2025:
            print(art)

        return articulos_2025

    except Exception as e:
        print(f"❌ Error al leer artículos de 2025: {e}")
        return []
    
articulos_encontrados = leer_articulos_2025(collection, client)

📦 Leyendo datos desde: mongo2:27017 (PRIMARY)
🔎 Se encontraron 20 artículos con versión creada en 2025:
{'titulo': 'Hamiltonian Graphs and the Traveling Salesman Problem', 'fecha_creacion': datetime.datetime(2025, 2, 25, 17, 59, 4)}
{'titulo': 'Proof of Riemann Hypothesis', 'fecha_creacion': datetime.datetime(2025, 3, 19, 3, 35, 48)}
{'titulo': 'The nature of electromagnetic energy', 'fecha_creacion': datetime.datetime(2025, 5, 14, 18, 20, 15)}
{'titulo': 'Periodic relativity: the theory of gravity in flat space time', 'fecha_creacion': datetime.datetime(2025, 1, 15, 6, 50, 35)}
{'titulo': 'Rationalization of EPR Coincidence Experiments', 'fecha_creacion': datetime.datetime(2025, 4, 1, 18, 36, 55)}
{'titulo': 'Generating functions for borders', 'fecha_creacion': datetime.datetime(2025, 3, 2, 18, 13, 43)}
{'titulo': 'Data Tables for Lorentz and CPT Violation', 'fecha_creacion': datetime.datetime(2025, 1, 13, 15, 37, 10)}
{'titulo': 'On Two Related Questions of Wilf Concerning Standard Y

Consulta 2

In [17]:
def buscar_articulos_csai_statml_3autores(collection):
    try:
        cursor = collection.find(
            {
                "$or": [
                    {"categories": {"$regex": r"cs\.AI"}},
                    {"categories": {"$regex": r"stat\.ML"}}
                ]
            },
            {"_id": 0, "title": 1, "authors": 1}
        ).limit(100)  # buscamos más para filtrar en Python

        resultados = []

        for doc in cursor:
            authors_str = doc.get("authors", "")
            # separar autores por coma o "and"
            autores = []
            if authors_str:
                # primero separar por comas
                partes = [a.strip() for a in authors_str.split(",")]
                # cada parte puede contener 'and', dividirlas también
                for parte in partes:
                    autores.extend([x.strip() for x in parte.split(" and ") if x.strip() != ""])

            if len(autores) >= 3:
                resultados.append({"title": doc.get("title"), "authors": autores})

            if len(resultados) == 10:
                break

        if resultados:
            print("Artículos en cs.AI o stat.ML con al menos 3 autores:")
            for art in resultados:
                print(f"- {art['title']}")
                print(f"  Autores: {art['authors']}")
                print("-" * 40)
        else:
            print("No se encontraron artículos con al menos 3 autores en las categorías dadas.")

        return resultados

    except Exception as e:
        print(f"Error al buscar artículos: {e}")
        return []

# Ejecutar función
articulos_encontrados = buscar_articulos_csai_statml_3autores(collection)



Artículos en cs.AI o stat.ML con al menos 3 autores:
- Calculating Valid Domains for BDD-Based Interactive Configuration
  Autores: ['Tarik Hadzic', 'Rune Moller Jensen', 'Henrik Reif Andersen']
----------------------------------------
- Personalizing Image Search Results on Flickr
  Autores: ['Kristina Lerman', 'Anon Plangprasopchok', 'Chio Wong']
----------------------------------------
- Unicast and Multicast Qos Routing with Soft Constraint Logic Programming
  Autores: ['Stefano Bistarelli', 'Ugo Montanari', 'Francesca Rossi', 'Francesco Santini']
----------------------------------------
- A study of structural properties on profiles HMMs
  Autores: ['Juliana S Bernardes', 'Alberto Davila', 'Vitor Santos Costa', 'Gerson\n  Zaverucha']
----------------------------------------
- Introduction to Arabic Speech Recognition Using CMUSphinx System
  Autores: ['H. Satori', 'M. Harti', 'N. Chenfour']
----------------------------------------
- Arabic Speech Recognition System using CMU-Sphin

Consulta 3

In [18]:
def leer_hep_ph_con_doi(collection):
    """
    Devuelve títulos, categorías y enlace PDF de artículos en 'hep-ph' con DOI.
    Limita a 15 resultados.
    """
    try:
        cursor = collection.find(
            {
                "categories": "hep-ph",
                "doi": {"$exists": True, "$ne": ""}
            },
            {
                "_id": 0,
                "title": 1,
                "categories": 1,
                "pdf_url": 1
            }
        ).limit(15)

        resultados = []
        for doc in cursor:
            resultados.append({
                "titulo": doc.get("title", "Sin título"),
                "categorias": doc.get("categories", []),
                "pdf_url": doc.get("pdf_url", "No disponible")
            })

        if resultados:
            print("Artículos en hep-ph con DOI:")
            for art in resultados:
                print(f"Título: {art['titulo']}")
                print(f"Categorías: {art['categorias']}")
                print(f"PDF: {art['pdf_url']}")
                print("-" * 40)
        else:
            print("No se encontraron artículos en hep-ph con DOI.")

        return resultados

    except Exception as e:
        print(f"Error al buscar artículos hep-ph con DOI: {e}")
        return []

articulos_encontrados = leer_hep_ph_con_doi(collection)

Artículos en hep-ph con DOI:
Título: Calculation of prompt diphoton production cross sections at Tevatron and
  LHC energies
Categorías: hep-ph
PDF: No disponible
----------------------------------------
Título: Lifetime of doubly charmed baryons
Categorías: hep-ph
PDF: No disponible
----------------------------------------
Título: Understanding the Flavor Symmetry Breaking and Nucleon Flavor-Spin
  Structure within Chiral Quark Model
Categorías: hep-ph
PDF: No disponible
----------------------------------------
Título: Crystal channeling of LHC forward protons with preserved distribution in
  phase space
Categorías: hep-ph
PDF: No disponible
----------------------------------------
Título: Probing non-standard neutrino interactions with supernova neutrinos
Categorías: hep-ph
PDF: No disponible
----------------------------------------
Título: Experimental efforts in search of 76Ge Neutrinoless Double Beta Decay
Categorías: hep-ph
PDF: No disponible
-------------------------------------

Consulta 4

In [19]:
def leer_articulos_con_doi(collection):
    """
    Devuelve títulos, autores y referencia de publicación de artículos con DOI.
    Ordena alfabéticamente por título y limita a 20 resultados.
    """
    try:
        cursor = collection.find(
            {"doi": {"$exists": True, "$ne": ""}},
            {
                "_id": 0,
                "title": 1,
                "authors": 1,
                "journal-ref": 1
            }
        ).sort("title", 1).limit(20)

        resultados = []
        for doc in cursor:
            resultados.append({
                "titulo": doc.get("title", "Sin título"),
                "autores": doc.get("authors", []),
                "referencia_publicacion": doc.get("journal-ref", "No disponible")
            })

        if resultados:
            print("Artículos con DOI:")
            for art in resultados:
                print(f"Título: {art['titulo']}")
                print(f"Autores: {art['autores']}")
                print(f"Referencia publicación: {art['referencia_publicacion']}")
                print("-" * 40)
        else:
            print("No se encontraron artículos con DOI.")

        return resultados

    except Exception as e:
        print(f"Error al buscar artículos con DOI: {e}")
        return []

articulos_encontrados = leer_articulos_con_doi(collection)

Artículos con DOI:
Título: !-Graphs with Trivial Overlap are Context-Free
Autores: Aleks Kissinger (University of Oxford), Vladimir Zamdzhiev (University
  of Oxford)
Referencia publicación: EPTCS 181, 2015, pp. 16-31
----------------------------------------
Título: !Qu\'e maravilla! Multimodal Sarcasm Detection in Spanish: a Dataset and
  a Baseline
Autores: Khalid Alnajjar and Mika H\"am\"al\"ainen
Referencia publicación: None
----------------------------------------
Título: "$1k_F$" Singularities and Finite Density ABJM Theory at Strong Coupling
Autores: Oscar Henriksson and Christopher Rosen
Referencia publicación: None
----------------------------------------
Título: "$\mathbf{{\textit K^-}{\textit p}{\textit p}}$", a
  ${\overline{K}}$-Meson Nuclear Bound State, Observed in $^{3}{\rm He}({K^-},
  {\Lambda} p)n$ Reactions
Autores: J-PARC E15 collaboration, S. Ajimura, H. Asano, G. Beer, C. Berucci,
  H. Bhang, M. Bragadireanu, P. Buehler, L. Busso, M. Cargnelli, S. Choi, C.
  Curc